#The UNIFAC Method

For a multicomponent mixture, the activity coefficient is calculated by

$$
\ln \gamma_i = \ln \gamma^c_i + \ln \gamma^r_i 
$$

where $\ln \gamma^c_i$ is the combinatorial part and $\ln \gamma^r_i$ is the residual part.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Prausnitz case

# acetone and n-pentane
# CH3, CH2, CH3CO
R = np.array([0.9011, 0.6744, 1.6724])
Q = np.array([0.848, 0.540, 1.488])

temperature = 307.0 # [K]

a = np.array([[  0.0,   0.0, 476.4],
              [  0.0,   0.0, 476.4],
              [26.76, 26.76,   0.0]])
psi = np.exp(- a / temperature)

# Groups
n0 = np.array([1.0, 0.0, 1.0])
n1 = np.array([2.0, 3.0, 0.0])
N = np.array([n0, n1]) # acetone and pentane in this ORDER

x = np.array([0.047, 1.0 - 0.047]) # New composition!

##Combinatorial activity

In [3]:
def calculate_combinatorial_activity_log(x, R, Q, N):
    r = np.array([np.sum(n0 * R), np.sum(n1 * R)])
    q = np.array([np.sum(n0 * Q), np.sum(n1 * Q)])

    phi = r * x / np.sum(r * x)
    theta = q * x / np.sum(q * x)
    zz = 10
    l = 0.5 * zz * (r - q) - (r - 1.0) 
    ln_gamma_c = np.log(phi / x) + 0.5 * zz * q * np.log(theta / phi) \
               + l - (phi / x) * np.sum(x * l)
    return ln_gamma_c

In [4]:
ln_gamma_c = calculate_combinatorial_activity_log(x, R, Q, N)

print ln_gamma_c
print np.exp(ln_gamma_c)

[-0.05271726 -0.00010177]
[ 0.94864819  0.99989824]


##Residual activity

In [5]:
def calculate_residual_activity_log(psi):    
    # Reference solution
    # x_acetone = n0 / np.sum(n0)
    # x_pentane = n1 / np.sum(n1)
    X = (N.T / np.sum(N, axis=1)).T

    # theta_pentane = Q * x_pentane / np.sum(Q * x_pentane)
    # theta_acetone = Q * x_acetone / np.sum(Q * x_acetone)
    theta_ref = ((Q * X).T / np.sum(Q * X, axis=1))

    A = np.einsum('mi,mk', theta_ref, psi)
    B = np.einsum('mi,km', (theta_ref.T/A).T, psi) 
    ln_gamma_ref = Q * (1.0 - np.log(A) - B)

    X = np.dot(x, N) / np.sum(N.T * x)
    theta = Q * X / np.sum(Q * X)

    A = np.einsum('m,mk', theta, psi)
    B = np.einsum('m,km', (theta / A), psi) 
    ln_gamma = Q * (1.0 - np.log(A) - B)

    return np.sum(N * (ln_gamma - ln_gamma_ref), axis=1)

ln_gamma_r = calculate_residual_activity_log(psi)
print ln_gamma_r
print np.exp(ln_gamma_r)

[ 1.66056077  0.00534819]
[ 5.26226091  1.00536252]


In [6]:
def calculate_activity_unifac(x, R, Q, N, psi):
    ln_gamma_c = calculate_combinatorial_activity_log(x, R, Q, N)
    ln_gamma_r = calculate_residual_activity_log(psi)
    ln_gamma = ln_gamma_c + ln_gamma_r
    return np.exp(ln_gamma)

gamma = calculate_activity_unifac(x, R, Q, N, psi)

print np.log(gamma)
print gamma

[ 1.6078435   0.00524642]
[ 4.99203431  1.00526021]


In [7]:
obtained_gamma = gamma
expected_gamma = np.array([4.99, 1.005])

print 'Obtained activity is %s' % obtained_gamma
print 'Expected activity is %s' % expected_gamma

assert np.allclose(obtained_gamma, expected_gamma, rtol=1.0e-3), 'Obatined value did not match expected!'

Obtained activity is [ 4.99203431  1.00526021]
Expected activity is [ 4.99   1.005]


##Phase Equilibrium

$$
f_i^v = f_i^l
$$
where,
$$
f_i^l = x_i \gamma_i^l f_{i,\text{pure}}^l(T,P)
$$
and
$$
f_i^v = \phi_i^v y_i P
$$
if $\phi_i^v = 1$ (ideal gas)

$$
x_i \gamma_i^l P_{i,\text{sat}}(T) = y_i P
$$

Hence,

$$
x_{CH_3OH} \gamma_{CH_3OH}^l P_{CH_3OH, \text{sat}}(T) + (1 - x_{CH_3OH}) \gamma_{H_2O}^l P_{H_2O, \text{sat}}(T) = P
$$

$$
y_i = \frac{x_i \gamma_i^l P_{i, \text{sat}}(T)}{P}
$$

In [8]:
# Prof Philippi case
# water and methanol
# H2O, CH3OH
R = np.array([0.92, 1.4311])
Q = np.array([1.4 , 1.432 ])

# Groups
n0 = np.array([1.0, 0.0])
n1 = np.array([0.0, 1.0])
N = np.array([n0, n1])

x = np.array([0.8, 0.2])

temperature = 323.15 # [K]

a = np.array([[   0.0, 289.6],
              [-181.0,   0.0]])
psi = np.exp(- a / temperature)
gamma = calculate_activity_unifac(x, R, Q, N, psi)

# Check results
obtained_gamma = gamma
expected_gamma = np.array([1.0436, 1.4957])

print 'Obtained activity is %s' % obtained_gamma
print 'Expected activity is %s' % expected_gamma

assert np.allclose(obtained_gamma, expected_gamma, rtol=1.0e-3), 'Obatined value did not match expected!'

Obtained activity is [ 1.04362481  1.49569659]
Expected activity is [ 1.0436  1.4957]


In [9]:
x_water = np.linspace(0.00000001, 1.0, 500, endpoint=False)
x_methanol = 1.0 - x_water
xx = np.append([x_water], [x_methanol], axis=0).T

Psat_water = 92.50
Psat_methanol = 417.40

gamma = []
for x in xx:
    gamma.append(calculate_activity_unifac(x, R, Q, N, psi))
gamma = np.array(gamma) # Convert to numpy array

P = xx[:, 0] * gamma[:, 0] * Psat_water + xx[:, 1] * gamma[:, 1] * Psat_methanol
y = Psat_methanol * gamma[:, 1] * xx[:, 1] / P

x_met = xx[:, 1]
y_met = y
line1 = plt.plot(x_met, P, label='Bubble-point curve')
line2 = plt.plot(y_met, P, label='Dew-point curve')

ax = plt.gca()
ax.grid(True)
ax.axvline(0, color='black', lw=2)
ax.axhline(0, color='black', lw=2)
ax.axis([0.0, 1.0, 0.0, 500.0])
ax.legend(loc='upper left')
ax.set_xlabel(xlabel='Methane molar fraction [mol/mol]')
ax.set_ylabel(ylabel='Pressure [bar]')
plt.show()

##Selected Components

Let's choose $C_4H_10$ (Ethane) and $C_5 H_{12}$ (Pentane) as components in our system.

Ethane:

* 2 Groups of $CH_3$
* 2 Groups of $CH_2$

Pentane

* 2 Groups of $CH_3$
* 3 Groups of $CH_2$

By checking Table 1 from the class notes, we find out that $R_{CH_3} = 0.9011$, $Q_{CH_3} = 0.848$, $R_{CH_2} = 0.6744$  and $Q_{CH_2} = 0.540$. $CH_3$ belongs to the main group 1 and secondary group 1 and $CH_2$ belongs to the main group 1 and to the secondary group 2. For this case, the values for $a = 0$.

In [10]:
# My case
# ethane and pentane
# C2, C5

# 2 Groupd
R = np.array([0.9011, 0.6744]) # CH3, CH2
Q = np.array([0.848 , 0.540 ])

# Groups
n0 = np.array([2.0, 2.0])
n1 = np.array([2.0, 3.0])
N = np.array([n0, n1])

temperature = 300.0 # [K]

a = np.array([[0.0, 0.0],
              [0.0, 0.0]])
psi = np.exp(- a / temperature)
x_butane = np.linspace(0.0000000001, 1.0, 500, endpoint=False)
x_pentane = 1.0 - x_butane
xx = np.append([x_butane], [x_pentane], axis=0).T

Psat_butane = 1.58 # [bar]
Psat_pentane = 0.7267 # [bar]

gamma = []
for x in xx:
    gamma.append(calculate_activity_unifac(x, R, Q, N, psi))
gamma = np.array(gamma) # Convert to numpy array

P = xx[:, 0] * gamma[:, 0] * Psat_pentane + xx[:, 1] * gamma[:, 1] * Psat_butane
y = Psat_butane * gamma[:, 1] * xx[:, 1] / P

x_butane = xx[:, 1]
y_butane = y
line1 = plt.plot(x_butane, P, label='Bubble-point curve')
line2 = plt.plot(y_butane, P, label='Dew-point curve')

ax = plt.gca()
ax.grid(True)
ax.axvline(0, color='black', lw=2)
ax.axhline(0, color='black', lw=2)
ax.axis([0.0, 1.0, 0.6, 1.6])
ax.legend(loc='upper left')
ax.set_xlabel(xlabel='Butane molar fraction [mol/mol]')
ax.set_ylabel(ylabel='Pressure [bar]')
plt.show()